In [ ]:
%load_ext autoreload
%autoreload 2

# TUNED | RANDOM FOREST | LOW MULTICOLLINEARITY


In [ ]:
import sys
sys.path.append('../../')
import tokamakTK

import pydotplus
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from sklearn.tree import plot_tree
from sklearn.inspection import permutation_importance
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support

pd.set_option('display.max_columns', None)
path = "../../data/"
fig_path = "../../../../LATEX/Latex Images/"
plt.rc('font',family = 'serif')


In [ ]:
# Obtained from Optimization

min_subset_ids = pd.read_csv(path+"R_ids_alpha_0.6357.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# REMOVING SPHERICAL TOKAMAKS
#DB5 = DB5[~DB5.TOK.isin(['MAST', 'NSTX', 'START'])]


# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label"] = 1


print(
    f"{ round( (len(min_subset_ids)/len(DB5))*100     ,2)  }% of the data decreased alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids)/len(DB5))*100 ,2)  }% of the data did not decrease alpha_R"
)

In [ ]:
features = ['AUXHEAT', 'BEIMHD', 'BETASTAR', 'BT', 'CONFIG', 'DIVMAT', 'DWDIA', 'ECHMODE', 
            'ELMFREQ', 'ELMTYPE', 'EVAP', 'ICSCHEME', 'IP', 
            'LIMMAT', 'NEL', 'NESOL', 'NUSTAR', 'PECRH', 'PELLET', 'PFLOSS', 
            'PICRH', 'POHM', 'PRAD', 'PREMAG', 'Q95', 'RHOSTAR', 
            'TIV', 'TOK', 'TORQ', 'WALMAT', 'WFFORM', 'WFICFORM', 'ZEFF', 'ZEFFNEO']
"""
(array([0.86479592, 0.9319407 ]),
 array([0.77045455, 0.96309192]),
 array([0.81490385, 0.94726027]),
 array([ 440, 1436]))
"""
"""# --------------------------------------------------------------------------------------------
# Most important features
features = ['NUSTAR', 'RHOSTAR', 'Q95', 'WFFORM', 'BETASTAR']"""

"""
(array([0.80705882, 0.93314955]),
 array([0.77954545, 0.94289694]),
 array([0.79306358, 0.93799792]),
 array([ 440, 1436]))
"""

In [ ]:
DB5 = tokamakTK.clean_categorical_data(DB5)

# Needed to respectively clean each dtype
num_features = DB5[features].select_dtypes(include=['int', 'float']).columns.tolist()
cat_features = DB5[features].select_dtypes(include=['object']).columns.tolist()

data_num = DB5[num_features+["TOK","DATE"]]
data_cat = DB5[cat_features]

# Notice that for trees, it is not necessary to scale the features, but can improve the performance
"""
    It's a good practice to consider feature scaling as part of the overall data 
    preprocessing pipeline, especially if there are specific considerations 
    or requirements in your analysis.
"""
data_num = tokamakTK.clean_numerical_data(data_num, SS_scaling=False, UL_scale=True)

data_ = pd.concat([data_num,
                  (pd.concat([
                       DB5[["label"]], 
                       tokamakTK.encode_categorical_ohe(data_cat)
                      ], axis=1)
                  )],
                  axis=1
                 )

In [ ]:
X = data_.drop("label", axis=1)
y = data_["label"]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=71, stratify=y)

In [ ]:
rf = RandomForestClassifier(min_samples_split=2,  
                            min_samples_leaf=1,
                            min_impurity_decrease=0.0007,
                            n_estimators = 90,
                            max_depth = 23,
                            criterion='entropy',
                            max_features=None,
                            n_jobs=-1,
                            random_state=71
                           )

In [ ]:
rf.fit(X_train, y_train)
y_pred= rf.predict_proba(X_test)

In [ ]:
plt.figure(figsize=(5,4))

# keep probabilities for the positive outcome only
y_pred = y_pred[:, 1]
# calculate pr-curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
# plot the roc curve for the model
no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot(recall, precision, marker='.', label='Random Forest', color="r")
plt.plot([0,1], [no_skill,no_skill], linestyle='--', label='No Skill', color="k")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title("Low Multicollinearity Variables")
plt.legend();

In [ ]:
y_pred_ = rf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred_, labels=[1,0])

## PERMUTATION IMPORTANCE

In [ ]:
n_permutations = 200
scoring_metrics = ["precision", "recall"]
permutations = permutation_importance(rf, X_test, y_test, n_repeats=n_permutations, 
                                     scoring=scoring_metrics,
                                     random_state=71)
Precision, Recall = [pd.DataFrame(np.zeros((len(X.columns), 2)), 
                                  index=X.columns, columns=["mean", "std"])]*2

In [ ]:
for i in permutations["precision"].importances_mean.argsort()[::-1]:
    if permutations["precision"].importances_mean[i] - 2 * permutations["precision"].importances_std[i] > 0:
        Precision.loc[X.columns[i], "mean"] = np.round(permutations["precision"].importances_mean[i],4)
        Precision.loc[X.columns[i], "std"] = np.round(permutations["precision"].importances_std[i],4)
Precision.sort_values("mean", ascending=False, inplace=True)

for i in permutations["recall"].importances_mean.argsort()[::-1]:
    if permutations["recall"].importances_mean[i] - 2 * permutations["recall"].importances_std[i] > 0:
        Recall.loc[X.columns[i], "mean"] = np.round(permutations["recall"].importances_mean[i],4)
        Recall.loc[X.columns[i], "std"] = np.round(permutations["recall"].importances_std[i],4)
Precision.sort_values("mean", ascending=False, inplace=True)

In [ ]:
# Assuming you have two dataframes: Precision and Recall
fig, axes = plt.subplots(1, 2, figsize=(15, 5))  # Create a figure with two subplots

# First subplot: Precision
axes[0].bar(Precision.index[:amount], Precision['mean'][:amount], color="#A84547")
axes[0].errorbar(Precision.index[:amount], Precision['mean'][:amount], yerr=Precision['std'][:amount], 
                 fmt='.', capsize=4, color='k')
for x, y, err in zip(range(amount), Precision['mean'][:amount], Precision['std'][:amount]):
    axes[0].annotate(f'±{err:.2f}', xy=(x, y + err), xytext=(0, 4), textcoords='offset points',
                     ha='center', va='bottom', color='k')
axes[0].set_ylabel('Mean Value', fontsize=15)
axes[0].set_title('Precision', fontsize=17)
axes[0].set_xticks(Precision.index[:amount])
axes[0].set_xticklabels(Precision.index[:amount], rotation=45)
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)

# Second subplot: Recall
axes[1].bar(Recall.index[:amount], Recall['mean'][:amount], color="#A84547")
axes[1].errorbar(Recall.index[:amount], Recall['mean'][:amount], yerr=Recall['std'][:amount], 
                 fmt='.', capsize=4, color='k')
for x, y, err in zip(range(amount), Recall['mean'][:amount], Recall['std'][:amount]):
    axes[1].annotate(f'±{err:.2f}', xy=(x, y + err), xytext=(0, 4), textcoords='offset points',
                     ha='center', va='bottom', color='k')
#axes[1].set_ylabel('Mean Value')
axes[1].set_title('Recall', fontsize=17)
axes[1].set_xticks(Recall.index[:amount])
axes[1].set_xticklabels(Recall.index[:amount], rotation=45)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)

axes[0].tick_params(axis='both', which='major', labelsize=12, width=2)
axes[0].tick_params(axis='both', which='minor', labelsize=10, width=1)
axes[1].tick_params(axis='both', which='major', labelsize=12, width=2)
axes[1].tick_params(axis='both', which='minor', labelsize=10, width=1)

plt.tight_layout()  # Adjust spacing between subplots
plt.show()  # Display the figure
